In [1]:
import rospy
import numpy as np
import pickle

from pathlib import Path
from scikitMedicalRobot.utilities import generate_timedate_cache_file, ensure_dir
from scikitMedicalRobot.ros import tf2_helper, medical_robot_helper
from scikitMedicalRobot.calibration.ros_sample_tools import position_recorder
from scikitMedicalRobot.calibration.tool import tcp_calibration
from scikitMedicalRobot.algo import rigid_transform_3D

Definition: environment

In [2]:
A_marker_id = 9
A_rgba=(1.0, 0.0, 0.0, 1.0)
A_scale=(0.005, 0.005, 0.005)

B_marker_id = 99 
B_rgba=(0.0, 0.0, 1.0, 1.0)
B_scale=(0.005, 0.005, 0.005)

T_marker_id = 999
T_rgba=(0.0, 1.0, 0.0, 1.0)
T_scale=(0.005, 0.005, 0.005)

Definition: link name 

In [3]:
base_link_name = 'base_link'
ee_link_name = 'tool0'
tool_link_name = 'tool_link'
medical_image_link = 'medical_link'
tool_rotation_axis = '-y'

`scikitMedicalRobot` Toolkits

In [4]:
rospy.init_node('registeration')

th = tf2_helper()
mrh = medical_robot_helper(tool_link_name, tool_rotation_axis, ee_link_name, base_link_name, medical_image_link)

Point Pickup Tool

In [5]:
th.delete_all_markers()

In [6]:
file_name = Path('test_data/tool_tip_calibration/20230109_170514_740159_tool_tip_calibration_endmill.pickle')
with open(file_name, 'rb') as f:
    data = pickle.load(f)

tcp_cali_result, solution = tcp_calibration.tip2tip_optim(data)

tcp_cali_matrix = th.euler_to_matrix([0, 0, 0])
tcp_cali_matrix[0:3, 3] = np.array([tcp_cali_result[0, 0], tcp_cali_result[1, 0], tcp_cali_result[2, 0]])
position, quat = th.matrix_to_position_quat(tcp_cali_matrix)
th.pub_static_tf(position, quat, frame=tool_link_name, parent_frame=ee_link_name)

A: Points in Medical Coordinate System

In [ ]:
# points for registeration
y_offset = 0.025 * 7
x_offset = 0.025 * 9
screw_header_height = 0.008
A = np.array([
    [0, 0, screw_header_height],
    [0, y_offset, screw_header_height],
    [x_offset, y_offset, screw_header_height],
    [x_offset, 0, screw_header_height]
])

f_name = generate_timedate_cache_file(caches_dir='test_data', child_dir='position_recorder', filetype='pickle', suffix='points_in_medical')
ensure_dir(f_name.parent)
with open(f_name, "wb") as f:
    pickle.dump(np.array(A), f)
print(f"saved to {f_name}")

B: Points in Robot Coordinate System

In [ ]:
sampler = position_recorder(th, tool_link_name, name='points_in_robot', caches_dir='test_data', child_dir='position_recorder')
sampler.run()

Data Loading and Visualization

In [6]:
th.delete_all_markers()

In [8]:
# A
file_name = Path('test_data/position_recorder/20230310_152108_953470_points_in_medical.pickle')
with open(file_name, 'rb') as f:
    A = pickle.load(f)

for idx, position in enumerate(A):
    th.add_sphere_marker(position, marker_id=idx+A_marker_id, rgba=A_rgba, scale=A_scale)
    vis_pos = np.copy(position)
    vis_pos[0] = vis_pos[0] + 0.01
    th.add_text_marker(f'A{idx}', vis_pos, marker_id=idx+A_marker_id+100, rgba=A_rgba, text_size=A_scale[0]*1.2)

In [ ]:
# B
file_name = Path('test_data/position_recorder/20230310_152259_608782_points_in_robot.pickle')
with open(file_name, 'rb') as f:
    B = pickle.load(f)

for idx, position in enumerate(B):
    th.add_sphere_marker(position, marker_id=idx+B_marker_id, rgba=B_rgba, scale=B_scale)
    vis_pos = np.copy(position)
    vis_pos[0] = vis_pos[0] + 0.01
    th.add_text_marker(f'B{idx}', vis_pos, marker_id=idx+B_marker_id+100, rgba=B_rgba, text_size=B_scale[0]*1.2)

Rigidbody Transform 3D

In [ ]:
R, t = rigid_transform_3D(A.T, B.T)
T = np.eye(4)
T[0:3, 0:3] = R
T[0:3, 3] = t.flatten()

for idx, position in enumerate(A):
    new_position = R @ position + t.flatten()
    th.add_sphere_marker(new_position, marker_id=idx+T_marker_id, rgba=T_rgba, scale=T_scale)
    vis_pos = np.copy(new_position)
    vis_pos[0] = vis_pos[0] + 0.01
    th.add_text_marker(f'A{idx}->B{idx}', vis_pos, marker_id=idx+T_marker_id+100, rgba=T_rgba, text_size=T_scale[0]*1.2)

err = 0
for idx, position in enumerate(A):
    new_position = R @ position + t.flatten()
    err = err + np.linalg.norm(new_position - B[idx])

err = 1000 * err / len(A)
print(f'error: {err} mm')